In [1]:
import os
import pandas as pd
import datetime as dt
import time
import calendar
import numpy as np
from statistics import mean

from uce_resources import get_engine, get_site_id, get_mms_data, get_applied_forecast, get_prices, get_green_tariff
from uce_resources import make_results

In [2]:
sites_list = ['Myroliubivka', 'Novokondakove', 'Kyselivka', 'Poniativka', 'Bar', 'Verkhivka', 'Balky', 
              'Sharhorod_1', 'Kostohryzove', 'Chechelnyk_1', 'Chechelnyk_2', 'Stanislavchyk', 'Kopaihorod', 'Cherniatka', 
              'Bilozerka', 'Hlybochok_1', 'Hlybochok_2.1', 'Hlybochok_2.2', 'Mykolaivka', 'Komyshany_1', 'Komyshany_2', 
              'Dibrovka', 'Velihen', 'Veliton', 'Mala_Lepetykha', 'Rubanivka', 'Oleshky_2', 'Oleshky_1',
              'Afanasiivka', 'Bazaltova', 'Liubymivka', 'Balivka', 'Pohrebyshche', 'Bilashky', 'Yelanets_1',
              'Yelanets_2', 'Inhulets_1', 'Inhulets_2', 'Bereznehuvate', 'Teplychna', 'Solone', 'Stepnohirsk', 
              'Vasylivka', 'Kachkarivka', 'Hnatkiv', 'Porohy', 'Kulevcha', 'Shyroke']
# sites_list = ['Inhulets_1', 'Inhulets_2', 'Vasylivka']
sites_data = dict.fromkeys(sites_list)
print(len(sites_data.keys()))

48


In [3]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.pool import NullPool
from sqlalchemy.sql import select, and_

do_settings = {'host': 'db-postgresql-do-pv-do-user-1646238-0.db.ondigitalocean.com',
               'port': 25060,
               'database': 'pv-analytics', 
               'user': 'pv-analytics-master',
               'password': 'dzn9dyvnkp2llz3k'}

url = 'postgresql://{}:{}@{}:{}/{}'
url = url.format(do_settings['user'], do_settings['password'], do_settings['host'], do_settings['port'], do_settings['database'])
engine = create_engine(url, poolclass=NullPool)
metadata = MetaData()
metadata.reflect(bind=engine)

C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\dialects\postgresql\base.py:3802: SAWarning: Did not recognize type 'point' of column 'location'
  util.warn(


In [4]:
limitations_info = pd.read_excel('./data/solargis/models_enhancement/limitation_info.xlsx').to_dict(orient='list')
limitations_data = dict()
for site in limitations_info.keys():
    limitation_data = [record.date() for record in limitations_info[site] if not pd.isnull(record)]
    limitations_data.update({site: limitation_data})
print(len(limitations_data.keys()))

48


In [5]:
target_years = [2021]
target_months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

with engine.connect() as connection:
        
    for site in sites_data.keys():
        start = time.time()
        print('-'*50)
        print(site)
        site_data = dict()
        site_data['site'] = site
        site_data['site_id'], site_data['legal_entity'] = get_site_id(site, connection, 
                                                                      metadata.tables['sites'],
                                                                      include_legal_entity_id=True)
        mms_data_table = metadata.tables['mms_data']
        query = select([mms_data_table.c.timestamps_utc,
                mms_data_table.c.total_v1, mms_data_table.c.total_v2, mms_data_table.c.total_v3]).\
                where(and_(mms_data_table.c.site == site_data['site_id'], mms_data_table.c.year.in_(target_years), 
                mms_data_table.c.month.in_(target_months)))
        response = connection.execute(query).fetchall()
        # print(len(response[0]))
        mms_data_list = list()
        for response_row in response:
            if response_row[-1]:
                mms_data = pd.DataFrame(response_row[-1], index=response_row[0], columns=['yield'])
            elif response_row[-2]:
                mms_data = pd.DataFrame(response_row[-2], index=response_row[0], columns=['yield'])
            else:
                mms_data = pd.DataFrame(response_row[-3], index=response_row[0], columns=['yield'])
            mms_data.index.name = 'timestamp'
            mms_data_list.append(mms_data)

        base = dt.date(year=target_year, month=target_month, day=1)
        dates_range = [base + dt.timedelta(days=days) for days in range(calendar.monthrange(target_year, target_month)[-1])]
        forecasts_applied_table = metadata.tables['forecasts_applied']
        query = select([forecasts_applied_table.c.time_indexes_utc, forecasts_applied_table.c.energy], forecasts_applied_table.c.date.in_(dates_range)).\
                    where(and_(forecasts_applied_table.c.site == site_data['site_id'], forecasts_applied_table.c.update_version == 0))
        response = connection.execute(query).fetchall()
        forecast_data = pd.DataFrame(columns=['forecast'])
        for record in response:
            data = pd.DataFrame(record[1], index=record[0], columns=['forecast'])
            forecast_data = forecast_data.append(data)
        site_data['forecast_data'] = forecast_data.sort_index() * 1000
        site_data['forecast_data'] = site_data['forecast_data'].round(0).astype(int)

        site_data['mms_data'] = pd.concat(mms_data_list).sort_index()
        site_data['mms_data']['normal_operation_flag'] = 0
        for date in limitations_data[site]:
            site_data['mms_data']['normal_operation_flag'].loc[site_data['mms_data']['normal_operation_flag'].index.date == date] = 1
        print(site_data['mms_data']['normal_operation_flag'].sum(), len(limitations_data[site]))
        sites_data.update({site: site_data})
        site_data['mms_data'].to_excel('./results/yearly_data/yield_{}_{}-{}.xlsx'.\
            format(site.lower(), site_data['mms_data'].index.date.min().strftime('%Y%m%d'), site_data['mms_data'].index.date.max().strftime('%Y%m%d')))



--------------------------------------------------
Myroliubivka
1392 74
--------------------------------------------------
Novokondakove
2400 173
--------------------------------------------------
Kyselivka
2016 108
--------------------------------------------------
Poniativka
1656 93
--------------------------------------------------
Bar
768 46
--------------------------------------------------
Verkhivka
1080 54
--------------------------------------------------
Balky
864 50
--------------------------------------------------
Sharhorod_1
888 43
--------------------------------------------------
Kostohryzove
912 61
--------------------------------------------------
Chechelnyk_1
600 41
--------------------------------------------------
Chechelnyk_2
552 38
--------------------------------------------------
Stanislavchyk
888 49
--------------------------------------------------
Kopaihorod
1056 60
--------------------------------------------------
Cherniatka
816 51
-------------------------